In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import mysql.connector
from sqlalchemy import create_engine, text

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(1, '../')

import os

In [ ]:
def csv_load(ruta_carpeta):
   
    dataframes = []

    for archivo in os.listdir(ruta_carpeta):
        
        if archivo.endswith('.csv'):
            
            path = os.path.join(ruta_carpeta, archivo)

            df = pd.read_csv(path)
            
            dataframes.append((archivo.replace('.csv', ''), df))

    return dataframes

lista_dataframes = csv_load('../data')

names_df = []
dataframes = []

for nombre, df in lista_dataframes:
    names.append(nombre)
    dataframes.append(df)

In [ ]:
# funcion
